In [1]:
import torch
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### torch

In [2]:
class LinearRegression:
    def __init__(self, num_inputs, lr=0.001, batch_size=10, num_epochs=2000):
        self.w = torch.zeros((num_inputs, 1), requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)
        self.lr = lr
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.params = [self.w, self.b]

    def net(self, X):
        """线性回归模型"""
        return torch.matmul(X, self.w) + self.b

    def squared_loss(self, y_hat, y):
        """均方损失"""
        return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

    def sgd(self):
        """小批量随机梯度下降"""
        with torch.no_grad():
            for param in self.params:
                param -= self.lr * param.grad / self.batch_size
                param.grad.zero_()

    def fit(self, features, labels):
        """训练模型"""
        for epoch in range(self.num_epochs):
            for X, y in self.data_iter(self.batch_size, features, labels):
                l = self.squared_loss(self.net(X), y)
                l.sum().backward()
                self.sgd()
            with torch.no_grad():
                train_l = self.squared_loss(self.net(features), labels)
                if (epoch + 1) % 400 == 0:
                    print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

    @staticmethod
    def data_iter(batch_size, features, labels):
        """数据迭代器"""
        num_examples = len(features)
        indices = list(range(num_examples))
        random.shuffle(indices)
        for i in range(0, num_examples, batch_size):
            batch_indices = torch.tensor(
                indices[i: min(i + batch_size, num_examples)])
            yield features[batch_indices], labels[batch_indices]

### homework_ex1_01

In [3]:
data1 = pd.read_csv("../data/ex1data1.txt", names=["population", "profit"])
X1 = np.array(data1['population']).reshape(97, 1)
y1 = np.array(data1['profit']).reshape(97, 1)

In [4]:
features = torch.tensor(X1, dtype=torch.float32)
labels = torch.tensor(y1, dtype=torch.float32)

model = LinearRegression(num_inputs=features.shape[1])
model.fit(features, labels)

with torch.no_grad():
    train_l = model.squared_loss(model.net(features), labels)
    print(f'loss {float(train_l.mean()):f}')

epoch 400, loss 4.829054
epoch 800, loss 4.563023
epoch 1200, loss 4.498506
epoch 1600, loss 4.482086
epoch 2000, loss 4.478239
loss 4.478239


### homework_ex1_02

In [5]:
data2 = pd.read_csv('../data/ex1data2.txt', names=['size', 'bedroom', 'price'])
data2 = (data2 - data2.mean()) / data2.std()

X2 = np.array(data2.iloc[:, 0:-1])
y2 = np.array(data2.iloc[:, -1]).reshape((-1, 1))

In [6]:
features = torch.tensor(X2, dtype=torch.float32)
labels = torch.tensor(y2, dtype=torch.float32)

model = LinearRegression(num_inputs=features.shape[1])
model.fit(features, labels)

with torch.no_grad():
    train_l = model.squared_loss(model.net(features), labels)
    print(f'loss {float(train_l.mean()):f}')

epoch 400, loss 0.150276
epoch 800, loss 0.134400
epoch 1200, loss 0.131421
epoch 1600, loss 0.130832
epoch 2000, loss 0.130715
loss 0.130715
